## Step 1 - Data Engineering

In [216]:
# Import the Pandas library
import pandas as pd
import csv

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

Base = declarative_base()

In [217]:
#create an engine
engine = create_engine('sqlite:///hawaii.sqlite', echo=False)
# Create a connection to the engine called `conn`
conn = engine.connect()

In [218]:
measurement_df= pd.read_csv('./Clean_hawaii_measurements.csv')
station_df= pd.read_csv('./Clean_hawaii_stations.csv')

In [219]:
measurement_df.head()

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,4,USC00519397,2010-01-06,NaN,73


In [220]:
station_df.head()

,Unnamed: 0,Unnamed: 0.1,station,name,latitude,longitude,elevation
0,0,0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,1,1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,3,3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [221]:
#Creating Measurement and Station classes
class Measurement(Base):
    __tablename__ = "measurement"
    id = Column(Integer, primary_key = True)
    station = Column(String(255))
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Float)
    
class Station(Base):
    __tablename__ = "station"
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    location = Column(Float)

In [222]:
# Create the tables in the database
Base.metadata.create_all(engine)

In [223]:
# Use Orient='records' to create a list of data to write
measurement_data = measurement_df.to_dict(orient='records')
measurement_data[0]

{'Unnamed: 0': 0,
 'date': '2010-01-01',
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [224]:
# Use Orient='records' to create a list of data to write
station_data = station_df.to_dict(orient='records')
station_data[0]

{'Unnamed: 0': 0,
 'Unnamed: 0.1': 0,
 'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [225]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [226]:
# Save the reference to the `measurement`  and 'station' table as a variable called 'measurement_table' and 'station table'
measurement_table = sqlalchemy.Table('measurement', metadata, autoload=True)
station_table = sqlalchemy.Table('station', metadata, autoload=True)

In [227]:
conn.execute(measurement_table.insert(), measurement_data)

In [231]:
conn.execute(station_table.insert(), station_data)

In [232]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0)]

In [233]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0, None),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6, None),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0, None),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9, None),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6, None)]

In [230]:
#measurement_df.to_sql('measurement', engine, if_exists = "append", index=False)
#station_df.to_sql('station', engine, if_exists = "append", index=False)